*** TEAM NOTES ***
- be sure to update all code segements at the very end before submitting to ensure it's the most recent code pushed to GH

ENEL 645 Assignment 2
Animal Image Classification Model
Objectives Learned:
    - transfer learning using YoloV5 trained on Coco image dataset
    - implementation of convolutional neural networks (CNNs) using Pytorch
    - one-hot encoding
    - train validation test methods
Group 27:
    - Jose Eustaquio do Carmo Junior
    - Louis-Antoine Etchian
    - Michael Francis
    - Austyn Nagribiano
    - Peter Yuan
    - Mouri Zakir

CONTENTS
1. Introduction
2. data
3. ...


This project uses an animal classification dataset (https://www.kaggle.com/datasets/ashishsaxena2209/animal-image-datasetdog-cat-and-panda) with three classifiers: cats, dogs, pandas. The imageset contains 3000 labelled images evenly split between each of the three classes.


Importing libraries

In [21]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import os
import numpy as np
import glob
from sklearn.model_selection import StratifiedShuffleSplit
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
import matplotlib as mpl
import matplotlib.pyplot as plt

Getting data statistics:

'get_data_statistics()' returns the mean and standard deviation of the dataset.

In [22]:
def get_dataset_stats(data_loader):
    mean = 0.
    std = 0.
    nb_samples = 0.
    for data in data_loader:
        data = data[0]  # Get the images to compute the stgatistics
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        nb_samples += batch_samples

    mean /= nb_samples
    std /= nb_samples
    return mean, std

Defining a custom class for use with PyTorch framework:

'TorchVisionDataset()' is a custom dataset class to be used within the PyTorch framework. It inherits the 'Dataset' class from PyTorch and takes in two arguments: 'data_dic' and 'transform'. 'data_dic' is a dictionary with two keys each for the imageset and labels. 'transform' applies a specific transformation to the dataset with the default set to none.

Two methods are defined for this class: '__len__' and '__getitem__'. '__len__' returns the length of the imageset or the total number of samples. '__getitem__' is the main functionality for this class and returns the image and label for a defined index ('idx') input as the argument. The image is extracted from the dataset filepath and if a transform is specified, applied to the image before returning with the label.

In [23]:
class TorchVisionDataset(Dataset):
    def __init__(self, data_dic, transform=None):
        self.file_paths = data_dic["X"]
        self.labels = data_dic["Y"]
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        label = self.labels[idx]
        file_path = self.file_paths[idx]

        image = Image.open(file_path)

        if self.transform:
            image = self.transform(image)
        return image, label

Defining the model:

AnimalModel defines the model framework for the convolutional neural network (CNN). The mode takes in the number of classes, input shape of data, and a transfer learning flag as inputs. If transfer learning is activated, as in this project, the model obtains pre-trained weights from 'YoloV5' and freezes the validation layers.

Two methods are defined for this class: '_get_conv_output()' and 'forward()'. '_get_conv_output()' returns the number of features from the feature extractor, useful in initializing the final fully connected classification layer. 'forward()' makes the final pass of the data to the output. It applies the feature extractor to the input data, flattens, and then applies the final linear activation to classify the image.

In [24]:
class GarbageModel(nn.Module):
    def __init__(self, num_classes, input_shape, transfer=False):
        super().__init__()

        self.transfer = transfer
        self.num_classes = num_classes
        self.input_shape = input_shape

        # transfer learning if pretrained=True
        self.feature_extractor = models.resnet18(pretrained=transfer)

        if self.transfer:
            # layers are frozen by using eval()
            self.feature_extractor.eval()
            # freeze params
            for param in self.feature_extractor.parameters():
                param.requires_grad = False

        n_features = self._get_conv_output(self.input_shape)
        self.classifier = nn.Linear(n_features, num_classes)

    def _get_conv_output(self, shape):
        batch_size = 1
        tmp_input = torch.autograd.Variable(torch.rand(batch_size, *shape))

        output_feat = self.feature_extractor(tmp_input)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size

    # will be used during inference
    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

loading data

In [25]:
def get_data_loaders(images_path, val_split, test_split, batch_size=32, verbose=True):
    """
    These function generates the data loaders for our problem. It assumes paths are
    defined by "/" and image files are jpg. Each subfolder in the images_path
    represents a different class.

    Args:
        images_path (_type_): Path to folders containing images of each class.
        val_split (_type_): percentage of data to be used in the val set
        test_split (_type_): percentage of data to be used in the val set
        verbose (_type_): debug flag

    Returns:
        DataLoader: Train, validation and test data laoders.
    """

    # Listing the data
    # Cats
    print("LISTING DATA")
    # print(os.listdir("dataset"))
    input_dir = "dataset/temp/cats"
    # images = [Image.open(os.path.join(input_dir, image)) for image in os.listdir(input_dir)]  # load
    images = [os.path.join(input_dir, image) for image in os.listdir(input_dir)]
    cat_images = np.array(images)  # transform to numpy
    cat_labels = ['cat'] * len(cat_images)

    # Dogs
    input_dir2 = "dataset/temp/dogs"
    # images2 = [Image.open(os.path.join(input_dir2, image)) for image in os.listdir(input_dir2)]  # load
    images2 = [os.path.join(input_dir2, image) for image in os.listdir(input_dir2)]
    dog_images = np.array(images2)  # transform to numpy
    dog_labels = ['dog'] * len(dog_images)

    # Panda
    input_dir3 = "dataset/temp/panda"
    # images3 = [Image.open(os.path.join(input_dir3, image)) for image in os.listdir(input_dir3)]  # load
    images3 = [os.path.join(input_dir3, image) for image in os.listdir(input_dir3)]
    panda_images = np.array(images3)  # transform to numpy
    panda_labels = ['panda'] * len(panda_images)

    # Appending lists
    images = np.append(np.append(cat_images, dog_images), panda_images)
    labels = cat_labels + dog_labels + panda_labels
    print(len(labels))
    labels = np.array(labels)

    # Formatting the labs as ints
    classes = np.unique(labels).flatten()
    labels_int = np.zeros(labels.size, dtype=np.int64)

    # Convert string labels to integers
    for index, class_name in enumerate(classes):
        labels_int[labels == class_name] = index

    if verbose:
        print("Number of images in the dataset:", len(images))
        for index, class_name in enumerate(classes):
            print("Number of images in class ", class_name,
                  ":", (labels_int == index).sum())

    # Splitting the data in dev and test sets
    sss = StratifiedShuffleSplit(
        n_splits=1, test_size=test_split, random_state=10)
    sss.get_n_splits(images, labels_int)
    dev_index, test_index = next(sss.split(images, labels_int))

    dev_images = images[dev_index]
    dev_labels = labels_int[dev_index]

    test_images = images[test_index]
    test_labels = labels_int[test_index]

    # Splitting the data in train and val sets
    val_size = int(val_split * images.size)
    val_split = val_size / dev_images.size
    sss2 = StratifiedShuffleSplit(
        n_splits=1, test_size=val_split, random_state=10)
    sss2.get_n_splits(dev_images, dev_labels)
    train_index, val_index = next(sss2.split(dev_images, dev_labels))

    train_images = images[train_index]
    train_labels = labels_int[train_index]

    val_images = images[val_index]
    val_labels = labels_int[val_index]

    if verbose:
        print("Train set:", train_images.size)
        # print(train_labels)
        print("Val set:", val_images.size)
        # print(val_labels)
        print("Test set:", test_images.size)
        # print(test_labels)

    # Representing the sets as dictionaries
    train_set = {"X": train_images, "Y": train_labels}
    val_set = {"X": val_images, "Y": val_labels}
    test_set = {"X": test_images, "Y": test_labels}

    # Transforms
    torchvision_transform_train = transforms.Compose([transforms.Resize((640, 640)),
                                                      transforms.RandomHorizontalFlip(),
                                                      transforms.RandomVerticalFlip(),
                                                      transforms.ToTensor()])

    # Datasets
    train_dataset_unorm = TorchVisionDataset(
        train_set, transform=torchvision_transform_train)

    # Get training set stats
    trainloader_unorm = torch.utils.data.DataLoader(
        train_dataset_unorm, batch_size=batch_size, shuffle=True, num_workers=0)

    mean_train, std_train = get_dataset_stats(trainloader_unorm)

    if verbose:
        print("Statistics of training set")
        print("Mean:", mean_train)
        print("Std:", std_train)

    torchvision_transform = transforms.Compose([transforms.Resize((640, 640)),
                                                transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=mean_train, std=std_train)])

    torchvision_transform_test = transforms.Compose([transforms.Resize((640, 640)),
                                                     transforms.ToTensor(),
                                                     transforms.Normalize(mean=mean_train, std=std_train)])

    # Get the train/val/test loaders
    train_dataset = TorchVisionDataset(
        train_set, transform=torchvision_transform)
    val_dataset = TorchVisionDataset(val_set, transform=torchvision_transform)
    test_dataset = TorchVisionDataset(
        test_set, transform=torchvision_transform_test)

    trainloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    valloader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, num_workers=0)
    testloader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, num_workers=0)

    return trainloader, valloader, testloader

Train and validation

In [26]:
def train_validate(net, trainloader, valloader, epochs, batch_size,
                   learning_rate, best_model_path, device, verbose):
    best_loss = 1e+20
    for epoch in range(20):  # loop over the dataset multiple times

        # Loss function and optimizer
        criterion = nn.CrossEntropyLoss()  # Loss function
        optimizer = torch.optim.AdamW(net.parameters(), lr=learning_rate)
        scheduler = ExponentialLR(optimizer, gamma=0.9)

        # Training Loop
        train_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        print(f'{20 + 1},  train loss: {train_loss / i:.3f},', end=' ')
        scheduler.step()

        val_loss = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for i, data in enumerate(valloader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = net(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
            print(f'val loss: {val_loss / i:.3f}')

            # Save best model
            if val_loss < best_loss:
                print("Saving model")
                torch.save(net.state_dict(), best_model_path)
                best_loss = val_loss

    print('Finished Training')

testing

In [27]:
def test(net, testloader, device):
    correct = 0
    total = 0
    print(len(testloader))
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            print("images:")
            print(images[0].size())
            print(images[0].permute(1,2,0))
            image = torch.Tensor.cpu(images[0])
            plt.imshow(image.permute(1,2,0))
            print("labels:")
            print(labels)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            print("predicted:")
            print(predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


    print(
        f'Accuracy of the network on the test images: {100 * correct / total} %')

Main

In [ ]:
if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument('--learning_rate', type=float, default=1e-4, help='initial learning rate')
    parser.add_argument('--batch_size', type=int, default=1, help='batch size,  number of images in each iteration during training')
    parser.add_argument('--epochs', type=int, default=20, help='total epochs')
    parser.add_argument('--val_split', type=float, default=0.2, help='val split')
    parser.add_argument('--test_split', type=float, default=0.2, help='test split')
    parser.add_argument('--best_model_path', type=str, default="best_model", help='best model path')
    parser.add_argument('--images_path', type=str, default=1e-4, help='path to images')
    parser.add_argument('--verbose', type=bool, default=True, help='verbose debugging flag')
    parser.add_argument('--transfer_learning', type=bool, default=True, help='transfer learning flag')

    args = parser.parse_args()

    # Check if GPU is available
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Assuming that we are on a CUDA machine, this should print a CUDA device:
    if args.verbose:
      print(device)

    trainloader, valloader, testloader = \
    get_data_loaders(args.images_path, args.val_split, args.test_split,\
                     batch_size=args.batch_size, verbose=args.verbose)


    # Create our model
    net = GarbageModel(3, (3,640,640), args.transfer_learning)
    net.to(device)

    train_validate(net, trainloader, valloader, args.epochs, args.batch_size,\
         args.learning_rate, args.best_model_path, device, args.verbose)

    net.load_state_dict(torch.load(args.best_model_path))

    test(net, testloader,device)